[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/BouleJaune/picselliaT/blob/master/Object_Detection_TF1.ipynb)



# Object-detection made easy
We will learn how to easily train an object detection model from a list of pre-trained models with the dataset you created on the picsell-IA platform.

# Prerequisite

In [ ]:
!git clone https://github.com/BouleJaune/picselliaT
%cd picselliaT/
!pip install picsellia
%tensorflow_version 1.x

# Imports

In [ ]:
import sys
sys.path.append("slim")
from picsellia import Client
import picsell_utils
import tensorflow as tf

# Setting up your Picsell client


We need to connect with the Picsell platform to retrieve all of your project data. 

In [ ]:
api_token = "your_api_token"
project_token = "your_project_token" 
model_name = "your_model_name"

clt = Client(api_token=api_token)
clt.checkout_project(project_token=project_token)
clt.checkout_network(model_name)

Once the client is initialized we can start the pre-processing of our data.

# Data pre-processing

Most of the pre-processing is done by the Tensorflow object-detection API. However we do need to put our data in a TFRecord format for the API to understand it and to create a protobuf config file describing all the parameters of our model, input pipeline, evaluation and training configuration ...

Hopefully everything is handled by the picsell-utils package. 

## Downloading data and generating the label map

First we need the annotations and images on our machine. We also need to generate a label map, mapping the labels names to a label ID that the Tensorflow object-detection API can comprehend. We simply run the commands below to achieve this.

In [ ]:
clt.dl_annotations()
clt.generate_labelmap()
clt.train_test_split()
clt.dl_pictures()

## TFRecord files creation

Tensorflow needs the data in a TFRecord format, a memory efficient format storing data as a sequence of binary records. We generate two .record files, one for the training and one for the evaluation.
We set the annotation type to rectangle so we can extract the bounding boxes and their label from the annotations and store them inside the records.

In [ ]:
annotation_type = "rectangle" 
picsell_utils.create_record_files(label_path=clt.label_path, record_dir=clt.record_dir,
                         tfExample_generator=clt.tf_vars_generator, annotation_type=annotation_type)

## Editing the configuration file

Every trainable object-detection model downloaded from the Picsell.ia hub is provided with the protobuf configuration file used to train it. We want to edit this file and set some variables specific to our project. 

Most of those variables are provided by the SDK, but it's still up to you to set up some of them. 
If you want to dwell inside the configuration file to have more control over your model, you can still open it up with after that and go change some settings like the learning rate decay strategy. But it's not the purpose of this guide.

In [ ]:
#The number of steps you want your model to be trained on
nb_steps = 1000
#Your batch size, highly depending of your hardware and model architecture
#Some models will return errors while training with a really low batch size
batch_size = 16
#The learning rate used, it can be left to None to use the previous one. 
learning_rate = None

picsell_utils.edit_config(model_selected=clt.model_selected, 
            config_output_dir=clt.config_dir,
            record_dir=clt.record_dir, 
            label_map_path=clt.label_path, 
            num_steps=nb_steps,
            batch_size=batch_size, 
            learning_rate=learning_rate,
            annotation_type=annotation_type,
            eval_number = len(clt.eval_list))

## Training

Now that the input pipeline is built we can finally launch the training. To do this we can use the wrapper function train from picsell_utils , we simply specify where we will save the checkpoint and where the configuration file is located.

In [ ]:
picsell_utils.train(ckpt_dir=clt.checkpoint_dir, 
                     conf_dir=clt.config_dir)

Once the training is done we want to send the training logs to the Picsell.ia platform.

In [ ]:
dict_log = picsell_utils.tfevents_to_dict(path=clt.checkpoint_dir)
clt.send_logs(dict_log)

## Evaluating

Next is the evaluation phase, we can launch the evaluation with the evaluate method of the picsell_utils module.

In [ ]:
metrics = picsell_utils.evaluate(clt.metrics_dir, clt.config_dir, clt.checkpoint_dir)
clt.send_metrics(metrics)

You can send the checkpoints to the Picsell.ia platform now.

In [ ]:
clt.send_checkpoints()

# Exporting and infering

Now that the training is done and that we checked the performance of our model through the metrics returned by the evaluation we may want to export and use this model.

## Exporting the model

To export our model as a saved_model.pb we use the export_infer_graph function while specifying the right paths. We can send it to the platform to be used on the playground for live inference.

In [ ]:
picsell_utils.export_infer_graph(ckpt_dir=clt.checkpoint_dir, 
                       exported_model_dir=clt.exported_model_dir, 
                       pipeline_config_path=clt.config_dir)

clt.send_model()

## Inference

Lastly we want to use the model on some images of our evaluation set so we can send the results to the platform and see them on the dashboard.

In [ ]:
picsell_utils.infer(clt.eval_list, exported_model_dir=clt.exported_model_dir, 
      label_map_path=clt.label_path, results_dir=clt.results_dir)
clt.send_examples()